# Aceleración de Matching y Procesamiento de Grafos

Este notebook integra mejoras de paralelización, vectorización y uso opcional de CUDA (con CuPy) para acelerar la ejecución del código original sin alterar su lógica fundamental. Las mejoras se aplican a:

- **Vectorización**: Se elimina el bucle anidado en el cálculo de la matriz de coste en `enhanced_spatial_matching`.
- **Paralelización**: Se paralelizan los cálculos de hitting times y el procesamiento de pares en la evaluación de precisión.
- **Uso de CUDA (opcional)**: Se incluye una versión acelerada con CuPy para operaciones matriciales intensivas.

Asegúrate de tener instaladas las librerías necesarias (por ejemplo, `cupy`, `networkx`, `node2vec`, etc.) y de disponer de una GPU para aprovechar la aceleración por CUDA.

In [1]:
import os
import sys
import random
import logging

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.spatial import Delaunay
from scipy.optimize import linear_sum_assignment

import networkx as nx
import pandas as pd

import concurrent.futures



from node2vec import Node2Vec

# Agregar la ruta al DataLoader
sys.path.append(os.path.abspath('..'))
from data.dataloader import DataLoader

# Configuración del logger
logging.getLogger("gensim").setLevel(logging.WARNING)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO,
                    format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)
logger.info(f'Logging level: {logging.getLevelName(logger.getEffectiveLevel())}')

[2025-02-24 09:33:07] INFO: Logging level: INFO


In [ ]:
# Intentar importar CuPy para la versión CUDA (si está disponible)
print(sys.executable)

try:
    import cupy as cp
    HAS_CUPY = True
except ImportError:
    !{} -m pip install cupy
    import cupy as cp

c:\Users\jordi\anaconda3\python.exe


In [ ]:


from node2vec import Node2Vec

# Agregar la ruta al DataLoader
sys.path.append(os.path.abspath('..'))
from data.dataloader import DataLoader

# Configuración del logger
logging.getLogger("gensim").setLevel(logging.WARNING)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO,
                    format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)
logger.info(f'Logging level: {logging.getLevelName(logger.getEffectiveLevel())}')

In [2]:
# Cargar el dataset usando DataLoader
dl = DataLoader()
dlf = dl.load_data()
logger.info('Dataset cargado correctamente')

[2025-02-24 00:20:25] INFO: Dataset cargado correctamente


In [3]:
from PIL import Image

def load_and_preprocess_images(row1, row2, target_size=(512,512)):
    """
    Carga y redimensiona dos imágenes y ajusta los keypoints a un tamaño objetivo.
    """
    img_path1 = row1['img']
    img_path2 = row2['img']
    mat_path1 = row1['mat']
    mat_path2 = row2['mat']

    img1 = Image.open(img_path1)
    img2 = Image.open(img_path2)
    
    mat_data1 = sio.loadmat(mat_path1)
    mat_data2 = sio.loadmat(mat_path2)
    kpts1 = np.array(mat_data1['pts_coord'])
    kpts2 = np.array(mat_data2['pts_coord'])
    
    # Ajustar los keypoints al nuevo tamaño
    orig_size1 = img1.size
    orig_size2 = img2.size
    kpts1[0] = kpts1[0] * target_size[0] / orig_size1[0]
    kpts1[1] = kpts1[1] * target_size[1] / orig_size1[1]
    kpts2[0] = kpts2[0] * target_size[0] / orig_size2[0]
    kpts2[1] = kpts2[1] * target_size[1] / orig_size2[1]
    
    img1 = img1.resize(target_size, resample=Image.BILINEAR)
    img2 = img2.resize(target_size, resample=Image.BILINEAR)
    
    return img1, img2, kpts1, kpts2

def load_keypoints_from_row(row1, row2):
    mat_path1 = row1['mat']
    mat_path2 = row2['mat']

    kpts1 = np.array(sio.loadmat(mat_path1)['pts_coord'])   
    kpts2 = np.array(sio.loadmat(mat_path2)['pts_coord'])

    return kpts1, kpts2

def visualize_combined(img1, img2, kpts1, kpts2, adj_matrix1, adj_matrix2, matching):
    """
    Visualiza dos imágenes lado a lado con grafos de Delaunay y líneas que indican el matching.
    """
    h1, w1 = np.array(img1).shape[:2]
    h2, w2 = np.array(img2).shape[:2]

    composite_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    composite_img[:h1, :w1, :] = img1
    composite_img[:h2, w1:w1+w2, :] = img2

    # Desplazar keypoints de la segunda imagen
    kpts2_shifted = kpts2.copy()
    kpts2_shifted[0, :] += w1

    plt.figure(figsize=(10, 10))
    plt.imshow(composite_img)

    color = mcolors.CSS4_COLORS['yellow']

    # Dibujar grafos de Delaunay
    for kpts, adj_matrix in zip([kpts1, kpts2_shifted], [adj_matrix1, adj_matrix2]):
        N = kpts.shape[1]
        for i in range(N):
            for j in range(i+1, N):
                if adj_matrix[i, j]:
                    plt.plot([kpts[0, i], kpts[0, j]], [kpts[1, i], kpts[1, j]], '-', color=color, linewidth=1)
        plt.scatter(kpts[0], kpts[1], c=color, edgecolors='w', s=80)

    # Dibujar matching
    row_ind, col_ind = np.where(matching == 1)
    for r, c in zip(row_ind, col_ind):
        x1, y1 = kpts1[:, r]
        x2, y2 = kpts2_shifted[:, c]
        if r == c:
            plt.plot([x1, x2], [y1, y2], '-', color=mcolors.CSS4_COLORS['lime'], linewidth=1)
        else:
            plt.plot([x1, x2], [y1, y2], '-', color='r', linewidth=1)

    plt.title("Matching y Triangulación Delaunay")
    plt.axis("off")
    plt.show()

def delaunay_triangulation(kpts):
    """
    Construye la matriz de adyacencia usando Delaunay sobre los keypoints.
    """
    pts = kpts.T
    tri = Delaunay(pts)
    N = pts.shape[0]
    A = np.zeros((N, N))
    for simplex in tri.simplices:
        for i in range(len(simplex)):
            for j in range(i+1, len(simplex)):
                A[simplex[i], simplex[j]] = 1
                A[simplex[j], simplex[i]] = 1
    return A


In [4]:
def compute_node2vec_embeddings(G, dimensions=64, num_walks=10, walk_length=30):
    """
    Calcula embeddings usando node2vec sobre el grafo G.
    """
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=16, quiet=True)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    n = G.number_of_nodes()
    embeddings = np.zeros((n, dimensions))
    for node in G.nodes():
        embeddings[node] = model.wv[str(node)]
    return embeddings

def hitting_time(G, start_node, destination_node, num_walks=100):
    """
    Aproxima el hitting time entre dos nodos mediante caminatas aleatorias.
    """
    hits = []
    for _ in range(num_walks):
        current = start_node
        steps = 0
        while current != destination_node and steps < 1000:
            neighbors = list(G.neighbors(current))
            if not neighbors:
                break
            current = random.choice(neighbors)
            steps += 1
        hits.append(steps)
    return np.mean(hits)

def compute_hitting_time_matrix_parallel(G):
    """
    Computa la matriz de hitting times de forma paralela usando ProcessPoolExecutor.
    """
    n = G.number_of_nodes()
    hitting_times = np.zeros((n, n))

    def compute_pair(i, j):
        h_time = hitting_time(G, i, j)
        return (i, j, h_time)

    pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(compute_pair, i, j): (i, j) for i, j in pairs}
        for future in concurrent.futures.as_completed(futures):
            i, j, h_time = future.result()
            hitting_times[i, j] = h_time
            hitting_times[j, i] = h_time
    return hitting_times


In [5]:
def enhanced_spatial_matching(kpts1, kpts2, adj_matrix1, adj_matrix2):
    """
    Realiza matching utilizando características espaciales, embeddings (node2vec) y perfiles de hitting times.
    Esta versión vectoriza el cálculo de la matriz de coste.
    """
    n1 = kpts1.shape[1]
    n2 = kpts2.shape[1]
    
    # Construir grafos
    G1 = nx.from_numpy_array(adj_matrix1)
    G2 = nx.from_numpy_array(adj_matrix2)
    
    node2vec_emb1 = compute_node2vec_embeddings(G1)
    node2vec_emb2 = compute_node2vec_embeddings(G2)
    
    # Usar la versión paralela para hitting times (CPU)
    hitting_times1 = compute_hitting_time_matrix_parallel(G1)
    hitting_times2 = compute_hitting_time_matrix_parallel(G2)
    
    profile1 = np.mean(hitting_times1, axis=1)  # (n1,)
    profile2 = np.mean(hitting_times2, axis=1)  # (n2,)
    
    # Calcular distancias espaciales de forma vectorizada
    spatial_dist_matrix = np.linalg.norm(kpts1.T[:, np.newaxis, :] - kpts2.T[np.newaxis, :, :], axis=2)
    
    # Calcular distancias entre embeddings
    node2vec_dist_matrix = np.linalg.norm(node2vec_emb1[:, np.newaxis, :] - node2vec_emb2[np.newaxis, :, :], axis=2)
    
    # Diferencia en perfiles de hitting times
    hitting_profile_matrix = np.abs(profile1[:, np.newaxis] - profile2[np.newaxis, :])
    
    cost_matrix = 0.4 * spatial_dist_matrix + 0.3 * node2vec_dist_matrix + 0.3 * hitting_profile_matrix
    
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    matching = np.zeros((n1, n2), dtype=int)
    matching[row_ind, col_ind] = 1
    return matching

# Versión acelerada con CUDA usando CuPy (si está disponible)
def enhanced_spatial_matching_cuda(kpts1, kpts2, adj_matrix1, adj_matrix2):
    """
    Versión de enhanced_spatial_matching que utiliza CuPy para acelerar operaciones matriciales en GPU.
    """
    if not HAS_CUPY:
        raise ImportError("CuPy no está instalado o no se encontró una GPU disponible.")
        
    n1 = kpts1.shape[1]
    n2 = kpts2.shape[1]
    
    G1 = nx.from_numpy_array(adj_matrix1)
    G2 = nx.from_numpy_array(adj_matrix2)
    
    node2vec_emb1 = compute_node2vec_embeddings(G1)
    node2vec_emb2 = compute_node2vec_embeddings(G2)
    
    # Para hitting times se utiliza la versión CPU
    hitting_times1 = compute_hitting_time_matrix_parallel(G1)
    hitting_times2 = compute_hitting_time_matrix_parallel(G2)
    
    profile1 = np.mean(hitting_times1, axis=1)
    profile2 = np.mean(hitting_times2, axis=1)
    
    # Convertir a arrays de CuPy
    kpts1_cp = cp.asarray(kpts1.T)  # Shape: (n1, 2)
    kpts2_cp = cp.asarray(kpts2.T)  
    emb1_cp = cp.asarray(node2vec_emb1)
    emb2_cp = cp.asarray(node2vec_emb2)
    profile1_cp = cp.asarray(profile1)
    profile2_cp = cp.asarray(profile2)
    
    spatial_dist_matrix = cp.linalg.norm(kpts1_cp[:, cp.newaxis, :] - kpts2_cp[cp.newaxis, :, :], axis=2)
    node2vec_dist_matrix = cp.linalg.norm(emb1_cp[:, cp.newaxis, :] - emb2_cp[cp.newaxis, :, :], axis=2)
    hitting_profile_matrix = cp.abs(profile1_cp[:, cp.newaxis] - profile2_cp[cp.newaxis, :])
    
    cost_matrix = 0.4 * spatial_dist_matrix + 0.3 * node2vec_dist_matrix + 0.3 * hitting_profile_matrix
    
    # Transferir la matriz de coste a CPU para el algoritmo húngaro
    cost_matrix_cpu = cp.asnumpy(cost_matrix)
    row_ind, col_ind = linear_sum_assignment(cost_matrix_cpu)
    matching = np.zeros((n1, n2), dtype=int)
    matching[row_ind, col_ind] = 1
    return matching


In [6]:
def evaluate_matching_precision(kpts, matching):
    """
    Asume que la correspondencia ideal es la diagonal de la matriz de matching.
    """
    N = kpts.shape[1]
    row_ind, col_ind = np.where(matching == 1)
    correct = np.sum(row_ind == col_ind)
    return correct / N if N != 0 else 0

def compute_precision_for_all_categories(dlf, use_cuda=False):
    """
    Para cada categoría:
    1. Selecciona aleatoriamente un par de imágenes para visualizar el matching.
    2. Recorre todos los pares de imágenes (paralelizado) para calcular la precisión.
    3. Guarda los resultados globales en un CSV.
    """
    overall_results = []
    
    for cat_name, cat_df in dlf.items():
        if cat_name == '__pycache___df':
            continue
        
        # Visualización de un par aleatorio
        sample = cat_df.sample(2)
        row1 = sample.iloc[0]
        row2 = sample.iloc[1]
        
        img1, img2, kpts1, kpts2 = load_and_preprocess_images(row1, row2)
        adj1 = delaunay_triangulation(kpts1)
        adj2 = delaunay_triangulation(kpts2)
        
        if use_cuda and HAS_CUPY:
            matching = enhanced_spatial_matching_cuda(kpts1, kpts2, adj1, adj2)
        else:
            matching = enhanced_spatial_matching(kpts1, kpts2, adj1, adj2)
        
        logger.info(f"Visualizando par aleatorio para {cat_name}")
        visualize_combined(img1, img2, kpts1, kpts2, adj1, adj2, matching)
        
        # Paralelizar el cálculo de precisión para cada par de imágenes
        precisions = []
        n = len(cat_df)
        pairs = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
        
        def process_pair(pair):
            i, j = pair
            row_a = cat_df.iloc[i]
            row_b = cat_df.iloc[j]
            kpts_a, kpts_b = load_keypoints_from_row(row_a, row_b)
            A_a = delaunay_triangulation(kpts_a)
            A_b = delaunay_triangulation(kpts_b)
            if use_cuda and HAS_CUPY:
                matching_pair = enhanced_spatial_matching_cuda(kpts_a, kpts_b, A_a, A_b)
            else:
                matching_pair = enhanced_spatial_matching(kpts_a, kpts_b, A_a, A_b)
            return evaluate_matching_precision(kpts_a, matching_pair)
        
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = list(executor.map(process_pair, pairs))
        
        precisions.extend(results)
        
        media = np.mean(precisions)
        std = np.std(precisions)
        
        logger.info(f"Categoría {cat_name.capitalize()}: Precisión media = {media:.4f}, Desviación = {std:.4f}")
        logger.info(f"Resultados procesados para {cat_name} con {len(cat_df)} imágenes")
        
        overall_results.append({
            "Category": cat_name,
            "Mean_Accuracy": round(media, 4),
            "Std_Deviation": round(std, 4),
            "Number_of_Images": len(cat_df)
        })
    
    results_df = pd.DataFrame(overall_results)
    results_df.to_csv("results2.csv", index=False)
    logger.info("Resultados globales guardados en results2.csv")
    return results_df


In [9]:
if __name__ == "__main__":
    # Puedes definir si deseas usar CUDA estableciendo use_cuda=True (si cupy está instalado y hay GPU disponible)
    print("Usando CUDA:", HAS_CUPY)
    final_results = compute_precision_for_all_categories(dlf, use_cuda=True)
    
    # Mostrar resultados finales
    resultados = pd.read_csv("results2.csv")
    print(resultados)

Usando CUDA: False


AttributeError: Can't get local object 'compute_hitting_time_matrix_parallel.<locals>.compute_pair'